Copyright (c) Microsoft Corporation. All rights reserved.  

Licensed under the MIT License.

# ResNet50 Image Classification using ONNX

This example shows how to deploy the ResNet50 ONNX model as a web service using Azure Machine Learning services and the ONNX Runtime.

## What is ONNX
ONNX is an open format for representing machine learning and deep learning models. ONNX enables open and interoperable AI by enabling data scientists and developers to use the tools of their choice without worrying about lock-in and flexibility to deploy to a variety of platforms. ONNX is developed and supported by a community of partners including Microsoft, Facebook, and Amazon. For more information, explore the [ONNX website](http://onnx.ai).

## ResNet50 Details
ResNet classifies the major object in an input image into a set of 1000 pre-defined classes. For more information about the ResNet50 model and how it was created can be found on the [ONNX Model Zoo github](https://github.com/onnx/models/tree/master/models/image_classification/resnet). 

In [ ]:
import json
import time
import sys
import os
import numpy as np    # we're going to use numpy to process input and output data
import onnxruntime    # to inference ONNX models, we use the ONNX Runtime

import onnx
import glob
from onnx import numpy_helper

#### Download pre-trained ONNX model from ONNX Model Zoo.

Download the [ResNet50v2 model and test data](https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz) and extract it in the same folder as this tutorial notebook.


In [ ]:
import urllib.request

onnx_model_url = "https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz"
urllib.request.urlretrieve(onnx_model_url, filename="resnet50v2.tar.gz")

labels_url = "https://raw.githubusercontent.com/onnx/models/master/models/image_classification/synset.txt"
urllib.request.urlretrieve(labels, filename="synset.txt")

!tar xvzf resnet50v2.tar.gz

### Helper Functions

In [ ]:
with open("synset.txt", 'rb') as label_file:
    resnet_labels = np.array(label_file.read().splitlines())
    
resnet_labels

In [ ]:
def preprocess(img_data):
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    
    # for each pixel in each channel, divide the value by 255 
    # to get value between [0, 1] and then normalize
    
    for i in range(img_data.shape[0]):  
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
    return norm_img_data

def postprocess(result):
    prob = softmax(result)
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    return a

def softmax(x):
    x = np.array(x).reshape(-1)
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

### Inference for ResNet using ONNX Runtime

In [ ]:
session = onnxruntime.InferenceSession('resnet50v2/resnet50v2.onnx', None)

input_data_path = 'resnet50v2/test_data_set_0/input_0.pb'

In [ ]:
import onnx
from onnx import numpy_helper

# load in our data which is expected as NCHW 224x224 image
tensor = onnx.TensorProto()
with open(input_data_path, 'rb') as f:
    tensor.ParseFromString(f.read())
    
input_data = numpy_helper.to_array(tensor)

In [ ]:
%%time
input_name = session.get_inputs()[0].name  # get the id of the first input of the model
result = session.run([], {input_name: input_data})[0]

In [ ]:
np.take(resnet_labels,  postprocess(result)[:5])

In [ ]:
output_data_path = 'resnet50v2/test_data_set_0/output_0.pb'
tensor = onnx.TensorProto()
with open(output_data_path, 'rb') as f:
    tensor.ParseFromString(f.read())
    
output_data = numpy_helper.to_array(tensor)

In [ ]:
np.testing.assert_almost_equal(output_data, result, 4)

In [ ]:
np.take(resnet_labels,  postprocess(output_data)[:5])

In [ ]:
session = onnxruntime.InferenceSession('resnet50v2/resnet50v2.onnx', None)
test_data_dir = 'resnet50v2/test_data_set_0'

# Load inputs
inputs = []
inputs_num = len(glob.glob(os.path.join(test_data_dir, 'input_*.pb')))
for i in range(inputs_num):
    input_file = os.path.join(test_data_dir, 'input_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(input_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    inputs.append(numpy_helper.to_array(tensor))

# Load reference outputs
ref_outputs = []
ref_outputs_num = len(glob.glob(os.path.join(test_data_dir, 'output_*.pb')))
for i in range(ref_outputs_num):
    output_file = os.path.join(test_data_dir, 'output_{}.pb'.format(i))
    tensor = onnx.TensorProto()
    with open(output_file, 'rb') as f:
        tensor.ParseFromString(f.read())
    ref_outputs.append(numpy_helper.to_array(tensor))

# Run the model on the backend
input_name = session.get_inputs()[0].name  # get the id of the first input of the model
outputs = [session.run([], {input_name: inputs[i]})[0] for i in range(inputs_num)]

# Compare the results with reference outputs up to 4 decimal places
for ref_o, o in zip(ref_outputs, outputs):
    np.testing.assert_almost_equal(ref_o, o, 4)